# **Определение значимости признаков с использованием Random Forest и линейной регрессии**

Мы изучим, как оценить значимость признаков в нашем наборе данных с помощью двух популярных моделей: **Random Forest** и **линейной регрессии**. Это поможет нам понять, какие переменные наиболее влияют на время задержки поездов, и сделать более обоснованные выводы.

---

### **Шаг 1: Подготовка данных**

Прежде чем приступить к моделированию, нам нужно правильно подготовить данные, чтобы наши модели могли их интерпретировать. Мы закодируем категориальные переменные и разделим данные на обучающую и тестовую выборки.

```python
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Загрузка данных
df = pd.read_excel('railway_freight_data.xlsx')

# Кодирование категориальных признаков
le = LabelEncoder()
df['Cargo_Type'] = le.fit_transform(df['Cargo_Type'])
df['Departure_City'] = le.fit_transform(df['Departure_City'])
df['Arrival_City'] = le.fit_transform(df['Arrival_City'])

# Разделение данных на признаки и целевую переменную
# Удалим не количественные признаки и целевую переменную
X = df.drop(columns=['Delay_Hours', 'Date', 'Train_ID'])  # Признаки
y = df['Delay_Hours']  # Целевая переменная (задержка)

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
```

*Предложение:* **Перед тем как перейти к моделированию, подумайте, есть ли смысл нормализовать признаки, чтобы результаты были более интерпретируемыми, особенно для линейной регрессии.**

### **Шаг 2: Оценка значимости признаков с помощью Random Forest**

Random Forest – это мощный инструмент, который может помочь нам понять, какие признаки наиболее важны.

```python
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

# Создание и обучение модели Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Определение значимости признаков
feature_importances = rf_model.feature_importances_

# Визуализация значимости признаков
plt.figure(figsize=(10, 6))
plt.barh(X.columns, feature_importances, color='skyblue')
plt.xlabel('Важность признака')
plt.ylabel('Признак')
plt.title('Значимость признаков по модели Random Forest')
plt.show()
```

*Предложение:* **Попробуйте интерпретировать: если “Weight_Tons” оказался важным признаком, это может указывать на то, что тяжёлые грузы чаще вызывают задержки.**

По сути, мы построили модель прогнозирования задержки поездов в зависимости от его параметров.

```python
line=3 #для какой строки датасета делаем прозноз
print('Параметры маршрута:\n', X.loc[line])
print('---------')
print('Предсказанное значение:', rf_model.predict([X.loc[line].values])[0])
print('Фактическое значение:', y[line])
```

### **Шаг 3: Оценка значимости признаков с помощью линейной регрессии**

Линейная регрессия поможет нам понять, какие признаки влияют на время задержки и в каком направлении.

```python
from sklearn.linear_model import LinearRegression

# Создание и обучение модели линейной регрессии
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Определение значимости признаков (по коэффициентам)
coefficients = lr_model.coef_

# Визуализация значимости признаков
plt.figure(figsize=(10, 6))
plt.barh(X.columns, coefficients, color='lightgreen')
plt.xlabel('Коэффициент признака')
plt.ylabel('Признак')
plt.title('Значимость признаков по модели линейной регрессии')
plt.show()
```

Предсказание происходит аналогичным образом, как и в Random Forest.

*Предложение:* **Обратите внимание на знаки коэффициентов: если у признака положительный коэффициент, он увеличивает задержку, а отрицательный – уменьшает. Подумайте, почему так может происходить.**

### **Блок: Прогнозирование времени задержки и оценка точности моделей**

В этом блоке мы рассмотрим, как использовать обученные модели для прогнозирования времени задержки поездов и оценить их точность на обучающих и тестовых данных, используя основные метрики регрессионного анализа.

---

### **Шаг 1: Прогнозирование времени задержки**

После того как мы обучили модели Random Forest и линейной регрессии, можем использовать их для предсказания времени задержки на тестовых данных.

```python
# Прогнозирование на обучающих данных
y_train_pred_rf = rf_model.predict(X_train)
y_train_pred_lr = lr_model.predict(X_train)

# Прогнозирование на тестовых данных
y_test_pred_rf = rf_model.predict(X_test)
y_test_pred_lr = lr_model.predict(X_test)
```

### **Шаг 2: Оценка точности моделей**

Для оценки качества предсказаний можно использовать метрики, такие как **средняя абсолютная ошибка (MAE)**, **среднеквадратичная ошибка (MSE)** и **коэффициент детерминации (R²)**.

```python
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Оценка для модели Random Forest
mae_rf = mean_absolute_error(y_test, y_test_pred_rf)
mse_rf = mean_squared_error(y_test, y_test_pred_rf)
r2_rf = r2_score(y_test, y_test_pred_rf)

# Оценка для модели линейной регрессии
mae_lr = mean_absolute_error(y_test, y_test_pred_lr)
mse_lr = mean_squared_error(y_test, y_test_pred_lr)
r2_lr = r2_score(y_test, y_test_pred_lr)

# Вывод метрик
print("Метрики для модели Random Forest:")
print(f"MAE: {mae_rf:.2f}")
print(f"MSE: {mse_rf:.2f}")
print(f"R²: {r2_rf:.2f}")

print("\nМетрики для модели линейной регрессии:")
print(f"MAE: {mae_lr:.2f}")
print(f"MSE: {mse_lr:.2f}")
print(f"R²: {r2_lr:.2f}")
```

---

### **Объяснение метрик:**
1. **Средняя абсолютная ошибка (MAE)**: Показывает среднее отклонение предсказаний от фактических значений. Чем меньше MAE, тем лучше модель предсказывает.
2. **Среднеквадратичная ошибка (MSE)**: Указывает на среднюю квадратичную разницу между предсказанными и фактическими значениями. MSE более чувствителен к большим ошибкам, чем MAE.
3. **Коэффициент детерминации (R²)**: Показывает, какая доля дисперсии целевой переменной объясняется моделью. Значение R² ближе к 1 указывает на лучшую модель.

### **Шаг 3: Интерпретация результатов**

- **Сравните метрики между моделями:** Если одна модель показывает значительно более низкие ошибки (MAE и MSE) и высокое значение R², она считается более точной.
- **Проанализируйте недостатки:** Если обе модели показывают высокие ошибки, возможно, данные нуждаются в дополнительной обработке, или нужно добавить более информативные признаки.

*Предложение:* **После оценки точности подумайте о возможных улучшениях: добавлении новых признаков, использовании более сложных моделей или выборе других методов нормализации данных.**

Этот блок поможет вам понять, насколько хорошо ваши модели справляются с задачей прогнозирования времени задержки и какие улучшения можно внести в процесс анализа.

### **Шаг 4: Практическое задание**

*Предложение:* **Попробуйте убрать признаки с наименьшей значимостью, переобучите модели и сравните их производительность на тестовой выборке. Улучшились ли результаты? Что это говорит о важности этих признаков?**

Также попробуйте добавить свои новые признаки. Как изменилось качество можделей?

### **Заключение**

Сравнив значимость признаков по двум моделям, вы сможете лучше понять, какие переменные влияют на задержку поездов и как использовать эту информацию для оптимизации процессов. 

*Предложение:* **Помните, что модели – это инструменты для выявления инсайтов, но интерпретация этих инсайтов в реальном бизнес-контексте может потребовать дополнительных данных и анализа.** 

Этот подход поможет вам развивать навыки анализа данных и делать более обоснованные выводы на основе результатов моделей.